In [4]:
#make-training-data.ipynb
#
#by Joe Hahn
#joe.hahn@oracle.com
#3 July 2018
#
#generate training data

In [5]:
#number of dots in training dataset
N_train = 5#15000

#half-thickness of the x
x_half_width = 0.5

#radius of the O
radius = 3.5

#box half-width before 45 degree rotation
box_half_width = 7.1

#set jitter=scale of the gaussian noise, to make the class boundaries slightly fuzzy
jitter = 0.0

#set seed for random number generator
rn_seed = 13

#set debug=True to see debugging output
debug =  False

#generate the xo dataset
print 'generating xo data...'
initial_id = 0
from helper_fns import *
train = make_xo_data(N_train, initial_id, x_half_width, radius, box_half_width, jitter, rn_seed, debug)
print 'number of training records = ', len(train)

##save training data as csv file
#import pandas as pd
#pd.set_option('display.expand_frame_repr', False)
#print train.head(5)
#train.to_csv('data/train.txt', sep='|', index=False, header=False)

generating xo data...
number of training records =  5


In [6]:
train

,id,ran_num,class,Xscore,Oscore,Bscore,x0,y0,x,y
0,0,0.300906,B,0.0,0.0,1.0,-4.823464,-0.891648,-4.823464,-0.891648
1,1,0.787947,B,0.0,0.0,1.0,-2.352056,5.039304,-2.352056,5.039304
2,2,0.203496,B,0.0,0.0,1.0,4.860233,1.974452,4.860233,1.974452
3,3,0.068962,B,0.0,0.0,1.0,-0.631747,-3.611439,-0.631747,-3.611439
4,4,0.302051,O,0.0,1.0,0.0,-0.891088,2.861341,-0.891088,2.861341
